In [8]:
import numpy as np
import pandas as pd
import seaborn as sns

In [15]:
datosML=pd.read_csv(r"C:\Users\Abraham\Desktop\Solo\Bootcamp\MLproyecto\Data\Processed\datosML.csv",index_col=False)
datosML

,MES,Resultado,COMERCIAL_NUM,TELEOP_NUM,tipodeobra
0,2,0,4.00,1,1
1,3,0,2.00,2,2
2,3,0,4.00,2,1
3,4,0,2.00,2,4
4,4,0,1.00,3,1
...,...,...,...,...,...
1332,4,0,5.00,1,1
1333,4,0,5.00,9,1
1334,4,0,5.00,1,1
1335,4,0,5.00,3,2


In [16]:
datosML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337 entries, 0 to 1336
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MES            1337 non-null   int64  
 1   Resultado      1337 non-null   int64  
 2   COMERCIAL_NUM  1337 non-null   float64
 3   TELEOP_NUM     1337 non-null   int64  
 4   tipodeobra     1337 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 52.4 KB


In [17]:
datosML.describe()

,MES,Resultado,COMERCIAL_NUM,TELEOP_NUM,tipodeobra
count,1337.00,1337.00,1337.00,1337.00,1337.00
mean,5.90,0.23,3.38,3.31,3.36
std,3.39,0.42,1.87,2.78,1.61
min,1.00,0.00,1.00,1.00,1.00
25%,3.00,0.00,2.00,1.00,2.00
50%,6.00,0.00,3.00,2.00,4.00
75%,9.00,0.00,5.00,6.00,5.00
max,12.00,1.00,7.00,11.00,5.00


In [18]:
datosML.corr()

,MES,Resultado,COMERCIAL_NUM,TELEOP_NUM,tipodeobra
MES,1.00,-0.02,0.04,-0.07,0.09
Resultado,-0.02,1.00,0.09,-0.02,-0.03
COMERCIAL_NUM,0.04,0.09,1.00,-0.03,-0.03
TELEOP_NUM,-0.07,-0.02,-0.03,1.00,-0.12
tipodeobra,0.09,-0.03,-0.03,-0.12,1.00


# Comprobación de equilibrio de target

In [19]:
datosML["Resultado"].value_counts()

0    1035
1     302
Name: Resultado, dtype: int64

In [20]:
#Dado que se se observa un desequilibrio en nuestro target, realizaremos un oversampling para solucionarlo

from imblearn.over_sampling import RandomOverSampler

oversampler = RandomOverSampler()
X_oversampled, y_oversampled = oversampler.fit_resample(datosML.drop("Resultado", axis=1), datosML["Resultado"])

# Creamos un DataFrame con el oversampling
datosoversampled = pd.DataFrame(X_oversampled, columns=datosML.drop("Resultado", axis=1).columns)
datosoversampled["Resultado"] = y_oversampled


In [21]:
y_oversampled.value_counts()

0    1035
1    1035
Name: Resultado, dtype: int64

# Pre modelos machine learning

In [22]:
from sklearn.model_selection import cross_val_score
from sklearn import model_selection

In [23]:
X= X_oversampled
y= y_oversampled

In [24]:

validation_size = 0.20
seed = 7
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,
                                                                    y,
                                                                    test_size=validation_size,
                                                                    random_state=seed)


## Fast check modelos

In [25]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

clf= LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions= clf.fit(X_train, X_test, y_train, y_test)

100%|██████████| 29/29 [00:03<00:00,  8.36it/s]


In [26]:
print(models)

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
RandomForestClassifier             0.79               0.80     0.80      0.79   
BaggingClassifier                  0.79               0.79     0.79      0.79   
ExtraTreesClassifier               0.79               0.79     0.79      0.78   
ExtraTreeClassifier                0.78               0.78     0.78      0.78   
LabelPropagation                   0.78               0.78     0.78      0.77   
LabelSpreading                     0.77               0.77     0.77      0.77   
XGBClassifier                      0.77               0.77     0.77      0.77   
DecisionTreeClassifier             0.76               0.77     0.77      0.76   
LGBMClassifier                     0.70               0.70     0.70      0.70   
KNeighborsClassifier               0.67               0.67     0.67      0.67   
NuSVC                       

print(models)

## MODELO 1: LOGISTIC REGRESION

In [15]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [16]:
prediction= model.predict(X_test)


In [18]:
accuracyregression=  model.score(X_test, y_test)
accuracyregression

0.5603864734299517

In [20]:
name='Logistic Regression'
kfold = model_selection.KFold(n_splits=10) #Parte los datos en 10 trozos para usar validación cruzada / cross validation
cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')

msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(cv_results)

[0.52409639 0.56024096 0.51204819 0.4939759  0.52409639 0.54216867
 0.61818182 0.52121212 0.58181818 0.49090909]


## MODELO 2: RANDOM TREE

In [21]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

dtr = DecisionTreeRegressor()
dtr.fit(X_train,y_train)

DecisionTreeRegressor()

In [22]:
dtrprediction= dtr.predict(X_test)

In [24]:
dtraccuracy= dtr.score (X_test, y_test)
dtraccuracy

0.3576595432606253

In [25]:
dtraccscores = cross_val_score(dtr, X, y, cv=5)
dtraccscores

array([-1.30123438, -0.84111007, -0.82230483, -0.03289735,  0.        ])

## MODELO 3: BAGGING

In [26]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier


dtc = DecisionTreeClassifier()
modelbag = BaggingClassifier(base_estimator=dtc, n_estimators=100, random_state=seed)

In [27]:
modelbag.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100,
                  random_state=7)

In [28]:
bagpred = modelbag.predict(X_test)
bagcuracy=modelbag.score(X_test, y_test)
bagcuracy

0.782608695652174

In [29]:
bagscores = cross_val_score(modelbag, X, y, cv=5)
bagscores

array([0.71014493, 0.58937198, 0.77536232, 0.66666667, 0.72222222])

In [30]:
mean_score = bagscores.mean()
print("Mean Accuracy:", mean_score)

Mean Accuracy: 0.6927536231884057


In [31]:
# pruebas extra para validar la fiabilidad

from sklearn.metrics import precision_score, recall_score, f1_score

In [33]:
#precision 
precision = precision_score(y_test, bagpred)
precision

0.7327586206896551

In [35]:
#recall

recall = recall_score(y_test, bagpred)
recall

0.8585858585858586

In [37]:
#f1 score
f1 = f1_score(y_test, bagpred)
f1

0.7906976744186046

## MODELO 4: Ada Boost

In [38]:
from sklearn.ensemble import AdaBoostClassifier

adaboost = AdaBoostClassifier()
adaboost.fit(X_train, y_train)


AdaBoostClassifier()

In [39]:
adapred=adaboost.predict(X_test)


In [41]:
adaaccuracy=adaboost.score(X_test, y_test)
adaaccuracy

0.5893719806763285

In [42]:
adaboostscores = cross_val_score(adaboost, X, y, cv=5)
adaboostscores

array([0.4178744 , 0.3236715 , 0.50241546, 0.43478261, 0.48550725])

## MODELO 5: RANDOM FOREST

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)


In [47]:
rf_classifier.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [48]:
y_pred = rf_classifier.predict(X_test)


In [49]:
accuracy_randomforest = accuracy_score(y_test, y_pred)
accuracy_randomforest

0.7971014492753623

In [50]:
#Cross validation
forestscores = cross_val_score(rf_classifier, X, y, cv=5)
forestscores

array([0.69323671, 0.62318841, 0.75362319, 0.69565217, 0.71014493])

In [37]:
mean_forest=forestscores.mean()
mean_forest

0.6951690821256038

## MEJOR MODELO: RANDOM FOREST

In [51]:
import joblib

joblib.dump(rf_classifier, 'new_model.pkl')

['new_model.pkl']

# FEATURE IMPORTANCE

In [52]:
#feature importance


# Obtener la importancia de las características
importances = rf_classifier.feature_importances_

# Crear un DataFrame con las características y su importancia
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': importances})

# Ordenar las características por su importancia descendente
feature_importances = feature_importances.sort_values('Importance', ascending=False)

# Imprimir el ranking de importancia de las características
print(feature_importances)


         Feature  Importance
0            MES        0.37
2     TELEOP_NUM        0.26
1  COMERCIAL_NUM        0.20
3     tipodeobra        0.17


In [53]:
#importancia relativa de variables

importances = rf_classifier.feature_importances_
indices = np.argsort(importances)[::-1]

for i in range(len(indices)):
    print(f"Variable: {X.columns[indices[i]]} | Importancia: {importances[indices[i]]}")


Variable: MES | Importancia: 0.3671932545246347
Variable: TELEOP_NUM | Importancia: 0.264311086684645
Variable: COMERCIAL_NUM | Importancia: 0.19603768804451585
Variable: tipodeobra | Importancia: 0.17245797074620456
